In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
proj_dir = Path.cwd().parent.parent.parent


In [106]:
# load compiled_results_filtered.csv
# col_type_dict = {
#     "sampler_seed": int,
#     "RandomForestClassifier_max_depth": int,
#     "RandomForestClassifier_min_samples_leaf": int,
#     "RandomForestClassifier_min_samples_split": int,
#     "RandomForestClassifier_n_estimators": int,
#     "KNeighborsClassifier_n_neighbors": int,
#     "LogisticRegression_max_iter": int,
#     "SVC_degree": int,
#     "XGB_max_depth": int,
#     "XGB_scale_pos_weight": int,	
#     "XGB_seed": int,
# }

df = pd.read_csv('compiled_results.csv',)
df.head()

,uo_method,scaler_method,imbalance_ratio,classifier,sampler_seed,feat_col_list,date_time,dataset,precision_score_min,precision_score_max,...,SGDClassifier_loss,SGDClassifier_penalty,RandomForestClassifier_bootstrap,RandomForestClassifier_class_weight,RandomForestClassifier_criterion,RandomForestClassifier_max_depth,RandomForestClassifier_min_samples_leaf,RandomForestClassifier_min_samples_split,RandomForestClassifier_n_estimators,SVC_degree
0,smote,minmax,0.5,svm,9811435,"['smcdc__kurtosis', 'smcac__kurtosis', 'vib_sp...",2022-05-04-1150-40,milling,0.079619,0.092184,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,random_under,minmax,0.7,svm,24004803,"['smcdc__kurtosis', 'smcac__kurtosis', 'vib_sp...",2022-05-04-1150-40,milling,0.077713,0.096035,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,adasyn,minmax,0.8,ridge,597198,"['smcdc__kurtosis', 'smcac__kurtosis', 'vib_sp...",2022-05-04-1150-40,milling,0.098446,0.125964,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,smote,minmax,1.0,xgb,29209865,"['smcdc__kurtosis', 'smcac__kurtosis', 'vib_sp...",2022-05-04-1150-40,milling,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,minmax,0.1,knn,23585256,"['smcdc__kurtosis', 'smcac__kurtosis', 'vib_sp...",2022-05-04-1150-40,milling,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df.columns

Index(['uo_method', 'scaler_method', 'imbalance_ratio', 'classifier',
       'sampler_seed', 'feat_col_list', 'date_time', 'dataset',
       'precision_score_min', 'precision_score_max', 'precision_score_avg',
       'precision_score_std', 'recall_score_min', 'recall_score_max',
       'recall_score_avg', 'recall_score_std', 'f1_score_min', 'f1_score_max',
       'f1_score_avg', 'f1_score_std', 'rocauc_min', 'rocauc_max',
       'rocauc_avg', 'rocauc_std', 'prauc_min', 'prauc_max', 'prauc_avg',
       'prauc_std', 'accuracy_min', 'accuracy_max', 'accuracy_avg',
       'accuracy_std', 'n_thresholds_min', 'n_thresholds_max', 'SVC_C',
       'SVC_kernel', 'SVC_max_iter', 'SVC_verbose', 'RidgeClassifier_alpha',
       'XGB_alpha', 'XGB_eta', 'XGB_eval_metric', 'XGB_lambda',
       'XGB_max_depth', 'XGB_objective', 'XGB_scale_pos_weight', 'XGB_seed',
       'KNeighborsClassifier_metric', 'KNeighborsClassifier_n_neighbors',
       'KNeighborsClassifier_weights', 'LogisticRegression_class_wei

In [55]:
mill_feature_path = proj_dir / "data/processed/milling/milling_features.csv.gz"
df_feat = pd.read_csv(mill_feature_path, compression='gzip')
df_feat.head()

,cut_id,ae_spindle__length,ae_spindle__kurtosis,ae_table__length,ae_table__kurtosis,vib_spindle__length,vib_spindle__kurtosis,vib_table__length,vib_table__kurtosis,smcdc__length,smcdc__kurtosis,smcac__length,smcac__kurtosis,cut_no,case,tool_class
0,50_23,64.0,-0.536421,64.0,1.023564,64.0,0.052618,64.0,-1.182274,64.0,-0.741543,64.0,-1.379993,50,4,1
1,50_24,64.0,0.580646,64.0,-0.248433,64.0,-0.477432,64.0,-1.185010,64.0,-0.906486,64.0,-1.373383,50,4,1
2,50_25,64.0,-0.335575,64.0,-0.117931,64.0,0.385026,64.0,-1.320485,64.0,-1.051426,64.0,-1.398157,50,4,1
3,50_26,64.0,-1.180998,64.0,-0.227941,64.0,0.198078,64.0,-1.123297,64.0,-0.680110,64.0,-1.348922,50,4,1
4,50_27,64.0,-0.074406,64.0,-0.431231,64.0,-0.430192,64.0,-1.300146,64.0,-0.387458,64.0,-1.363613,50,4,1


In [56]:
from src.models.random_search_setup import (
    rf_params,
    xgb_params,
    knn_params,
    lr_params,
    sgd_params,
    ridge_params,
    svm_params,
    nb_params,
    general_params
)

general_params.keys()

dict_keys(['scaler_method', 'uo_method', 'imbalance_ratio', 'classifier'])

In [57]:
general_params_single = {}
for k in general_params.keys():
    general_params_single[k] = [df.iloc[0][k]]

In [58]:
# make a dict comprehension for general_params_single
general_params_single = {k: [df.iloc[0][k]] for k in general_params.keys()}
general_params_single

{'scaler_method': ['minmax'],
 'uo_method': ['random_over'],
 'imbalance_ratio': [0.5],
 'classifier': ['rf']}

In [84]:
# find all columns in df that begin with "RandomForestClassifier"
prestring = 'RandomForestClassifier'
clf_cols = {c.replace(f"{prestring}_",""): df.iloc[0][c]  for c in df.iloc[0].dropna().index if c.startswith(prestring)}
clf_cols

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 58.0,
 'min_samples_leaf': 7.0,
 'min_samples_split': 5.0,
 'n_estimators': 382.0}

In [85]:
# convert any whole numbers in clf_cols to int
for k in clf_cols.keys():
    if isinstance(clf_cols[k], float) and clf_cols[k].is_integer():
        clf_cols[k] = int(clf_cols[k])


In [86]:
clf_cols

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 58,
 'min_samples_leaf': 7,
 'min_samples_split': 5,
 'n_estimators': 382}

In [102]:
def rebuild_params_clf(df, row_idx):
    classifier_string = df.iloc[row_idx]['classifier']
    if classifier_string == "rf":
        prefix = 'RandomForestClassifier'

    elif classifier_string == "xgb":
        prefix = 'XGB'

    elif classifier_string == "knn":
        prefix = 'KNeighborsClassifier'

    elif classifier_string == "lr":
        prefix = 'LogisticRegression'

    elif classifier_string == "sgd":
        prefix = 'SGDClassifier'

    elif classifier_string == "ridge":
        prefix = 'RidgeClassifier'

    elif classifier_string == "svm":
        prefix = 'SVC'

    elif classifier_string == "nb":
        prefix = 'GaussianNB'

    params_clf = {c.replace(f"{prefix}_",""): df.iloc[row_idx][c]  for c in df.iloc[row_idx].dropna().index if c.startswith(prefix)}

    # convert any whole numbers in clf_cols to int
    for k in params_clf.keys():
        if isinstance(params_clf[k], float) and params_clf[k].is_integer():
            params_clf[k] = int(params_clf[k])

    return {k: [params_clf[k]] for k in params_clf.keys()} # put each value in a list

In [104]:
params_clf = rebuild_params_clf(df, 0)
params_clf

{'bootstrap': [False],
 'criterion': ['entropy'],
 'max_depth': [58],
 'min_samples_leaf': [7],
 'min_samples_split': [5],
 'n_estimators': [382]}